In [ ]:
from datasets import Dataset, load_dataset
import pandas as pd
import torch
import json
import os


In [ ]:
from unsloth import FastLanguageModel

# Optimal sequence length for medical prescriptions (short responses)
max_seq_length = 1024

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,  # Auto-detect best dtype
    load_in_4bit=True,  # 4-bit quantization for efficiency
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA rank - balanced for medical precision
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # Scaling factor (typically = r)
    lora_dropout = 0,  # 0 is optimized for Unsloth
    bias = "none",  # "none" is optimized
    use_gradient_checkpointing = "unsloth",  # 30% less VRAM
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)



In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma",  # Gemma-2 uses "gemma" template
)

In [ ]:
# Load mental health counseling data
dataset_raw = load_dataset("MaggiePai/mental_health_counseling_conversations", split="train")

# Print dataset info
print(f"Dataset: {dataset_raw}")
print(f"Number of examples: {len(dataset_raw)}")
print(f"Dataset features: {dataset_raw.features}")
print("\nFirst example:")
print(dataset_raw[0])

# Cut dataset to first 100 samples for faster training
dataset_raw = dataset_raw.select(range(100))
print(f"Dataset reduced to: {len(dataset_raw)} samples")
print(f"Dataset Length: {len(dataset_raw)}")


In [ ]:
# Convert to chat format for mental health counseling
def convert_to_chat_format(item):
    """Convert mental health counseling data to chat conversation format"""
    
    conversation = [
        {
            "role": "system",
            "content": "You are a compassionate and professional mental health counselor. Listen carefully to the client's concerns and provide empathetic, supportive, and constructive guidance."
        },
        {
            "role": "user",
            "content": item['Context']
        },
        {
            "role": "assistant",
            "content": item['Response']
        }
    ]
    
    return {"conversation": conversation}

# Convert all data to chat format
chat_data = [convert_to_chat_format(item) for item in dataset_raw]

print(f"Converted {len(chat_data)} conversations")
print("\nFirst conversation sample:")
print(chat_data[0])


In [ ]:
# Create Dataset from chat data
dataset = Dataset.from_list(chat_data)

print(f"Dataset size: {len(dataset)}")
print("\nFirst formatted conversation:")
print(dataset[0]['conversation'])

In [ ]:
# Format conversations with chat template
def formatting_prompts_func(examples):
    convos = examples["conversation"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)
print("Formatted prompt sample:")
print(dataset[0]['text'][:500] + "...")

In [ ]:
from trl import SFTTrainer, SFTConfig

# Optimal training configuration for mental health counseling data
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 50,
        num_train_epochs = 3,
        learning_rate = 2e-5,
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 42,
        output_dir = "../models/therapist-chatbot",
        save_strategy = "epoch",
        save_total_limit = 2,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        report_to = "none",
    ),
)

In [ ]:

# Train only on model responses (not user inputs or system prompts)
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

print("Training dataset preview:")
print(f"Total examples: {len(trainer.train_dataset)}")
print(f"Sample input_ids length: {len(trainer.train_dataset[0]['input_ids'])}")

In [ ]:
# Verify training setup - check what parts are being trained
sample_idx = 10
print("Full prompt:")
print(tokenizer.decode(trainer.train_dataset[sample_idx]["input_ids"]))
print("\n" + "="*80 + "\n")
print("Only training on (labels != -100):")
print(tokenizer.decode([x if x != -100 else tokenizer.pad_token_id for x in trainer.train_dataset[sample_idx]["labels"]]).replace(tokenizer.pad_token, ""))

In [ ]:
# Start training
print("🚀 Starting training...")
print(f"Total steps: ~{len(dataset) * 3 // (2 * 4)} steps (3 epochs, batch_size=2, grad_accum=4)")
print(f"Training on {len(dataset)} mental health counseling conversations")
print("Expected training time: 1-2 hours depending on GPU\n")

trainer_stats = trainer.train()

print("\n✅ Training completed!")
print(f"Final loss: {trainer_stats.training_loss:.4f}")

In [ ]:
# Enable fast inference mode
FastLanguageModel.for_inference(model)

def get_counseling_response(client_message, max_new_tokens=512, temperature=0.7):
    """Get counseling response from fine-tuned therapist model"""
    
    conversation = [
        {
            "role": "system",
            "content": "You are a compassionate and professional mental health counselor. Listen carefully to the client's concerns and provide empathetic, supportive, and constructive guidance."
        },
        {
            "role": "user",
            "content": client_message
        }
    ]
    
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    return response

# Test with mental health concerns
test_cases = [
    "I'm feeling really anxious about my future and I don't know what to do.",
    "I can't seem to get along with my family. We argue all the time.",
    "I've been feeling really depressed lately and nothing seems to help.",
    "My partner and I are having communication problems.",
    "I'm struggling with low self-esteem and confidence.",
]

print("💬 Testing Fine-tuned Therapist Chatbot\n" + "="*80 + "\n")

for concern in test_cases:
    print(f"CLIENT: {concern}")
    response = get_counseling_response(concern)
    print(f"THERAPIST: {response}")
    print("\n" + "-"*80 + "\n")

In [ ]:
# Save the fine-tuned model
print("💾 Saving fine-tuned model...")

# Save LoRA adapter
model.save_pretrained("../models/therapist-chatbot-lora")
tokenizer.save_pretrained("../models/therapist-chatbot-lora")

print("✅ LoRA adapter saved to ../models/therapist-chatbot-lora")
